In [1]:
import pandas as pd
import datetime
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
from random import randint as rn
import numpy as np
from sklearn.metrics import r2_score

### Підготовча частина

Завантаження підготовленої таблиці даних

In [2]:
df = pd.read_pickle('data.pickle')
y_true = df.Value.copy()
df.reset_index(inplace=True, drop=True)
df_res = df.copy()

Видалення випадковим чином заданого відсотка значень

In [3]:
def count_nan(df_nan):
    value = list(df_nan.isna().sum().loc[df_nan.isna().sum() > 0].sort_values(ascending=True))
    if len(value) > 0:
        return value[0]
    else:
        return 0

def make_nan(df, share=0.2):
    target = np.ceil(len(df) * 0.2)
    while count_nan(df) < target:
        position = rn(0, len(df) - 1)
        df.at[position, 'Value'] = np.nan
    return df

df_nan = make_nan(df, share=0.2)
nan_index = list(df_nan.loc[df_nan.Value.isna()].index.copy())

Перевірка кількості видалених значень у датасеті

In [4]:
print('Отримана таблиця містить {} NaN'.format(count_nan(df_nan)))

Отримана таблиця містить 20 NaN


### Лінійна інтерполяція

Створюємо клас _LinearImputer_, який дозволить відновляти значенням шляхом кусочно - лінійної інтерполяції. У випадку, коли декілька сусідніх членів є NaN, вони при лінійній інтерполяції отримують однакові значення.

In [5]:
class LinearImputer:
    
    def __init__(self):
        self.X = None
        self.y = None
        self.res = None

    def neighbour(self, k, up=True):
        """
        метод, що шукає найближчі значення не NaN залишаєтся від методу найближчих сусідів
        """
        n = k
        if up:
            while np.isnan(self.y[n]):
                if n + 1 < len(self.y):
                    n += 1
                else:
                    return len(self.y) - 1, np.nan
            return np.abs(n - k), self.y[n]
        else:
            while np.isnan(self.y[n]):
                if n - 1 > - 1:
                    n -= 1
                else:
                    return 0, np.nan
            return np.abs(n - k), self.y[n]
        
    def line(self, x, x1, x2, y1, y2):
        return ((x - x1) / (x2 - x1)) * (y2 - y1) + y1
    
    def fit_transform(self, X, y):
        """
        Підставляє значення замість NaN, користуючись принципом лінійної інтерполяції
        """
        self.X = np.array(X)
        self.y = np.array(y)
        self.res = np.array(y)
        for n in range(len(self.y)):
            if np.isnan(y[n]):
                shift_minus, y_minus = self.neighbour(n, up=False)
                shift_plus, y_plus = self.neighbour(n, up=True)
                x_minus, x_plus = X[n - shift_minus], X[n + shift_plus]
                self.res[n] = self.line(self.X[n], x_minus, x_plus, y_minus, y_plus)
        return self.res        

Проводимо відновлення значень

In [6]:
li = LinearImputer()
y_pred = li.fit_transform(df_nan.Time, df_nan.Value)
df['LI_imputed'] = y_pred

In [7]:
print('Коефіцієнт детермінації = {}'.format(r2_score(np.array(y_true)[nan_index], np.array(y_pred)[nan_index])))

Коефіцієнт детермінації = 0.9703729937404088
